In [118]:
import pandas as pd

data = pd.read_csv("FINALE.csv")
print(data.columns)

Index(['ETER_ID_year', 'ETER_ID', 'institution_name',
       'english_institution_name', 'reference_year', 'country_code',
       'name_of_the_city', 'latitude', 'longitude',
       'personnel_expenditure(PPP)', 'non-personnel_expenditure(PPP)',
       'expenditure_unclassified(PPP)', 'total_current_expenditure(PPP)',
       'capital_expenditure_(PPP)', 'total_core_budget(PPP)',
       'total_third_party_funding(PPP)', 'student_fees_funding(PPP)',
       'revenue_unclassified(PPP)', 'total_current_revenues(PPP)',
       'total_academic_staff(FTE)', 'total_staff_(FTE)',
       'lowest_degree_delivered', 'highest_degree_delivered',
       'total_students_enrolled_at_ISCED_5',
       'total_students_enrolled_at_ISCED_6',
       'total_students_enrolled_at_ISCED_7',
       'total_students_enrolled_ISCED_7_longdegree',
       'total_students_enrolled_ISCED_5-7', 'total_graduates_at_ISCED_5',
       'total_graduates_at_ISCED_6', 'total_graduates_at_ISCED_7',
       'total_graduates_at_ISCED_

In [119]:
new_data = data.filter([
         
       'total_academic_staff(FTE)', 'total_staff_(FTE)',
       'total_students_enrolled_ISCED_5-7', 'total_graduates_ISCED_5-7',
       'erasmus_total_incoming_students', 'erasmus_total_outgoing_students',
        ] , axis=1)
print(new_data.head(2))

  total_academic_staff(FTE) total_staff_(FTE)  \
0                         m                 m   
1                         m                 m   

  total_students_enrolled_ISCED_5-7 total_graduates_ISCED_5-7  \
0                             26602                         m   
1                              2915                         m   

  erasmus_total_incoming_students erasmus_total_outgoing_students  
0                               m                               m  
1                               m                               m  


In [120]:
newdata = new_data.apply(lambda x: x.str.replace(',','.'))
newdata = newdata.replace(["m" , "#VALUE!" , "xr" , "c" , "a" , "xc" , "x" , "nc" , "s" , "NaN"] , 0)
print(newdata.head(2))

  total_academic_staff(FTE) total_staff_(FTE)  \
0                         0                 0   
1                         0                 0   

  total_students_enrolled_ISCED_5-7 total_graduates_ISCED_5-7  \
0                             26602                         0   
1                              2915                         0   

  erasmus_total_incoming_students erasmus_total_outgoing_students  
0                               0                               0  
1                               0                               0  


In [121]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_of_data = pca.fit_transform(newdata)
print(pca_of_data)

[[ 2.65114188e+04  1.96682016e+03]
 [ 2.90507427e+03  2.15520667e+02]
 [ 1.55566997e-25 -3.86230727e-22]
 ...
 [ 1.03938242e+03 -2.28667626e+02]
 [ 1.36664475e+03 -3.01080456e+02]
 [ 1.02502379e+03 -6.22555321e+02]]


In [141]:
pca_of_data.min(0)

array([-6.87338884e-26, -2.06223048e+04])

In [143]:
pca_of_data.max(0)

array([3023196.39372525,   95273.46345256])

In [122]:
from sklearn.cluster import MiniBatchKMeans
kmeans = MiniBatchKMeans(n_clusters=2, init="k-means++" )
y = kmeans.fit_transform(pca_of_data)

In [123]:
y.argmax(0)

array([2913, 2913])

In [124]:
pca_to_dataframe = pd.DataFrame(y , columns=["x" , "y"])
labels_todatafra = pd.DataFrame(kmeans.labels_ , columns=["label"])

In [125]:
pca_to_dataframe.index

RangeIndex(start=0, stop=17652, step=1)

In [126]:
final1 = pd.concat([pca_to_dataframe, labels_todatafra], axis=1 , ignore_index=False )

In [127]:
final1.head(10)

,x,y,label
0,17881.298925,22481.740435,0
1,40300.634788,1628.665558,1
2,43152.131823,4268.583413,1
3,43152.131823,4268.583413,1
4,43152.131823,4268.583413,1
5,42133.821703,3273.993571,1
6,18467.558066,54563.963206,0
7,18616.325080,21651.298758,0
8,43152.131823,4268.583413,1
9,42461.723973,3591.602594,1


In [128]:
toconcat = data.filter(["institution_name" , "ETER_ID" , "reference_year", "country_code"])
final = pd.concat([final1, toconcat], axis=1 , ignore_index=False )

In [129]:
final.head(1)

,x,y,label,institution_name,ETER_ID,reference_year,country_code
0,17881.298925,22481.740435,0,Université Nice - Sophia-Antipolis,FR0001,2017,FR


In [130]:
data1 = pd.read_csv("statistic_per_uni.csv")
data1 = data1.filter(["missing_perc","cons_perc"])

In [131]:
final3 = pd.concat([final , data1], axis=1 , ignore_index=False )
final3.to_csv("fusion_kmeans.csv" , index=False)

In [132]:
final3.head(1)

,x,y,label,institution_name,ETER_ID,reference_year,country_code,missing_perc,cons_perc
0,17881.298925,22481.740435,0,Université Nice - Sophia-Antipolis,FR0001,2017,FR,0.6,0.2


In [133]:
y.max(0)


array([2982079.25892687, 3020526.34284152])

In [134]:
y.min(0)

array([700.78674546,  38.37492564])

In [135]:
pca_to_dataframe2 = pd.DataFrame(pca_of_data , columns=["x" , "y"])

In [136]:
pca_final = pd.concat([pca_to_dataframe2, toconcat], axis=1 , ignore_index=False )
pca_final2 = pd.concat([pca_final, data1], axis=1 , ignore_index=False )

In [138]:
pca_final2.to_csv("pca.csv" , index=False)